# Plotly Dash Coal Terminal Analysis

## Project Description

The following codes generates a dashboard that displays the coal reclaimers' actual seven-hour rolling average of tonnes moved vs the nominal rate. 

The dashboard displays the actual seven-hour rolling average difference below the nominal rate of tonnes of coal moved per reclaimer. 

If the reclaimers' lines (representing the seven-hour rolling average of the amount of coal moved in tonnes below the nominal amount) peak above the threshold set, the reclaimer will need maintenance. 

The user can adjust the threshold line by toggling the threshold parameter.

In [ ]:
# Import packages
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_auth

import numpy as np
import pandas as pd

import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools

# Set login credentials
USERNAME_PASSWORD_PAIRS = [['data','analyst']]

# establish app
app=dash.Dash(external_stylesheets = [dbc.themes.BOOTSTRAP])

# Set login credentials
auth = dash_auth.BasicAuth(app,USERNAME_PASSWORD_PAIRS)
server = app.server

# Set app layout
app.layout = html.Div([

    # App header
    html.Div([html.H2('Coal Terminal Maintenance Analysis',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})]),

    # App descriptive subtitle
    html.H4('Select Maintenance Threshold',
            style = {'font-family':'Arial, Helvetica, sans-serif',
                     'textAlign':'center',
                     'padding-top':20,
                     'padding-bottom':5}),

    # Slider output
    html.Div(id = 'threshold', style = {'textAlign':'center',
                                        'font-family':'Arial, Helvetica, sans-serif',
                                        'padding-bottom':5,
                                        'fontSize': 16}),

    # Set threshold slider
    dcc.Slider(id = 'slider',
               min = 0,
               max = 20,
               value = 10,
               marks = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
               step = 1),

    # Line chart
    html.Div([dbc.Row([dbc.Col(dbc.Card(dcc.Graph(id = 'feature_graphic1'),
                                        body = True,color = "dark",outline = True))],
                      style = {'padding-top': 5,
                               'padding-bottom': 5}),
              dbc.Row([dbc.Col(dbc.Card(dcc.Graph(id = 'feature_graphic2'),
                                        body = True,color = "dark",outline = True))],
                      style = {'padding-top': 5,
                               'padding-bottom': 5}),
              dbc.Row([dbc.Col(dbc.Card(dcc.Graph(id = 'feature_graphic3'),
                                        body = True,color = "dark",outline = True))],
                      style = {'padding-top': 5,
                               'padding-bottom': 5}),
              dbc.Row([dbc.Col(dbc.Card(dcc.Graph(id = 'feature_graphic4'),
                                        body = True,color = "dark",outline = True))],
                      style = {'padding-top': 5,
                               'padding-bottom': 5}),
              dbc.Row([dbc.Col(dbc.Card(dcc.Graph(id = 'feature_graphic5'),
                                        body = True,color = "dark",outline = True))],
                      style = {'padding-top': 5,
                               'padding-bottom': 5})],
             style = {'font-family':'Arial, Helvetica, sans-serif',
                      'padding-top':20,
                      'padding-right':20,
                      'padding-bottom':20,
                      'margin-left':20}),

    # Instructions
    html.Div([html.H2('Instructions',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'}),

              html.Div(html.P(["The above dashboard displays the coal reclaimers' actual seven-hour rolling \
              average of tonnes moved vs the nominal rate. The dashboard displays the actual seven-hour rolling \
              average difference below the nominal rate of tonnes of coal moved per reclaimer. \
              If the reclaimers' lines (representing the seven-hour rolling average of the amount of coal moved \
              in tonnes below the nominal amount) peak above or equal to the maintenance threshold set, the \
              reclaimer will need maintenance. The user can adjust the maintenance threshold line by toggling the \
              maintenance threshold slider. Above each line chart, the number of hours the reclaimer is down \
              regarding the tonnage of coal moved is displayed and adjusts as the maintenance threshold slider \
              is toggled."]),
                       style = {'padding':30,
                                'font-family':'Arial, Helvetica, sans-serif',
                                'line-height':30,
                                'textAlign':'center',
                                'fontSize':20})]),

    # Final ending block
    html.Div([html.H2('',
                      style = {'padding':30,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})])],
    style={'margin':0})

# Set Slider output label
@app.callback(Output('threshold','children'),
              [Input('slider','value')])

def slider(slider):

    return str(slider)+'%'

# Return line plots and slider threshold indicator
@app.callback([Output('feature_graphic1','figure'),
               Output('feature_graphic2','figure'),
               Output('feature_graphic3','figure'),
               Output('feature_graphic4','figure'),
               Output('feature_graphic5','figure')],
              [Input('slider','value')])

def graphic1(slider):

    # Import data
    coal_data = pd.read_csv('Coal_Data/Comprehensive_Unpivoted_Data2.csv')
    # Filter data
    rl1 = coal_data.loc[coal_data['Machine'] == 'RL1']
    # Set X
    X = rl1['Dt1']
    # Set y
    y = rl1['Rolling Average %'] * 100

    # Set hours >= threshold count
    m_count1 = rl1.loc[rl1['Rolling Average %'] >= slider / 100]
    m_count1 = len(m_count1['Rolling Average %'])

    # Create vis
    data1 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        marker = {'color':'Blue'}),
                      layout = go.Layout(height = 100,
                                         paper_bgcolor = 'rgba(0,0,0,0)',
                                         plot_bgcolor = 'rgba(0,0,0,0)',
                                         margin = {'t': 40, 'b': 10, 'l': 20, 'r': 20}),
                      layout_yaxis_range = [0,20])

    # Hide x-axis
    data1.update_xaxes(visible = False, showticklabels = False)

    # Hide y-axis
    data1.update_yaxes(visible = False, showticklabels = False)

    # Set title
    data1.update_layout(title = "Rl1 - There are " + str(m_count1) + " hours >= to the set threshold",
                        title_font_size = 16, title_font_color = 'Black')

    # Set horizontal line
    data1.add_hline(y = slider,
                    line = {'color': 'Red'},
                    line_dash = 'dash',
                    line_width = 2)

    ##############################################################################################################

    # Filter data
    rl2 = coal_data.loc[coal_data['Machine'] == 'RL2']
    # Set x
    X = rl2['Dt1']
    # Set y
    y = rl2['Rolling Average %'] * 100

    # Set hours >= threshold count
    m_count2 = rl2.loc[rl2['Rolling Average %'] >= slider / 100]
    m_count2 = len(m_count2['Rolling Average %'])

    # Create vis
    data2 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        marker = {'color':'Orange'}),
                      layout = go.Layout(height = 100,
                                         paper_bgcolor = 'rgba(0,0,0,0)',
                                         plot_bgcolor = 'rgba(0,0,0,0)',
                                         margin = {'t': 40, 'b': 10, 'l': 20, 'r': 20}),
                      layout_yaxis_range = [0,20])

    # Hide x-axis
    data2.update_xaxes(visible = False, showticklabels = True)

    # Hide y-axis
    data2.update_yaxes(visible = False, showticklabels = False)

    # Set title
    data2.update_layout(title = "Rl2 - There are " + str(m_count2) + " hours >= to the set threshold",
                        title_font_size = 16, title_font_color = 'Black')

    # Set horizontal line
    data2.add_hline(y = slider,
                    line = {'color': 'Red'},
                    line_dash = 'dash',
                    line_width = 2)

    ##############################################################################################################

    # Filter data
    sr1 = coal_data.loc[coal_data['Machine'] == 'SR1']
    # Set X
    X = sr1['Dt1']
    # Set y
    y = sr1['Rolling Average %'] * 100

    # Set hours >= threshold count
    m_count3 = sr1.loc[sr1['Rolling Average %'] >= slider / 100]
    m_count3 = len(m_count3['Rolling Average %'])

    # Create vis
    data3 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        marker = {'color':'Purple'}),
                      layout = go.Layout(height = 100,
                                         paper_bgcolor = 'rgba(0,0,0,0)',
                                         plot_bgcolor = 'rgba(0,0,0,0)',
                                         margin = {'t': 40, 'b': 10, 'l': 20, 'r': 20}),
                      layout_yaxis_range = [0,20])

    # Hide x-axis
    data3.update_xaxes(visible = False, showticklabels = False)

    # Hide y-axis
    data3.update_yaxes(visible = False, showticklabels = False)

    # Set title
    data3.update_layout(title = "SR1 - There are " + str(m_count3) + " hours >= to the set threshold",
                        title_font_size = 16, title_font_color = 'Black')

    # Set horiontal line
    data3.add_hline(y = slider,
                    line = {'color': 'Red'},
                    line_dash = 'dash',
                    line_width = 2)

    ##############################################################################################################

    # Filter data
    sr4a = coal_data.loc[coal_data['Machine'] == 'SR4A']
    # Set X
    X = sr4a['Dt1']
    # Set y
    y = sr4a['Rolling Average %'] * 100

    # Set hours >= threshold count
    m_count4 = sr4a.loc[sr4a['Rolling Average %'] >= slider / 100]
    m_count4 = len(m_count4['Rolling Average %'])

    # Create vis
    data4 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        marker = {'color':'mediumaquamarine'}),
                      layout = go.Layout(height = 100,
                                         paper_bgcolor = 'rgba(0,0,0,0)',
                                         plot_bgcolor = 'rgba(0,0,0,0)',
                                         margin = {'t': 40, 'b': 10, 'l': 20, 'r': 20}),
                      layout_yaxis_range = [0,20])

    # Hide x-axis
    data4.update_xaxes(visible = False, showticklabels = False)

    # Hide y-axis
    data4.update_yaxes(visible = False, showticklabels = False)

    # Set title
    data4.update_layout(title = "SR4A - There are " + str(m_count4) + " hours >= to the set threshold",
                        title_font_size = 16, title_font_color = 'Black')

    # Set horizontal line
    data4.add_hline(y = slider,
                    line = {'color': 'Red'},
                    line_dash = 'dash',
                    line_width = 2)

    ##############################################################################################################

    # Filter data
    sr6 = coal_data.loc[coal_data['Machine'] == 'SR6']
    # Set X
    X = sr6['Dt1']
    # Set y
    y = sr6['Rolling Average %'] * 100

    # Set hours >= threshold count
    m_count5 = sr6.loc[sr6['Rolling Average %'] >= slider / 100]
    m_count5 = len(m_count5['Rolling Average %'])

    # Create vis
    data5 = go.Figure(data = go.Scatter(x = X,
                                        y = y,
                                        marker = {'color':'Green'}),
                      layout = go.Layout(height = 100,
                                         paper_bgcolor = 'rgba(0,0,0,0)',
                                         plot_bgcolor = 'rgba(0,0,0,0)',
                                         margin = {'t': 40, 'b': 10, 'l': 20, 'r': 20}),
                      layout_yaxis_range = [0,20])

    # Hide x-axis
    data5.update_xaxes(visible = False, showticklabels = False)

    # Hide y-axis
    data5.update_yaxes(visible = False, showticklabels = False)

    # Set title
    data5.update_layout(title = "SR6 - There are " + str(m_count5) + " hours >= to the set threshold",
                        title_font_size = 16, title_font_color = 'Black')

    # Set horizntal line
    data5.add_hline(y = slider,
                    line = {'color': 'Red'},
                    line_dash = 'dash',
                    line_width = 2)

    return data1, data2, data3, data4, data5

if __name__ == '__main__':
    app.run_server()
